In [ ]:
# Create a server that runs at given times (9pm, 6am), every hour
# To check for which instances are running and give info on them
# on both AWS and GCP (and other)
# Everyday at 9pm let know the cost of running and number of instances
# Every hour, give a notification if the cost is over $x

In [108]:
import boto3, requests, json
from datetime import datetime, timezone
import time as pytime, pytz
from dgnutils import notify
ec2 = boto3.client('ec2')
pricing = boto3.client('pricing') # create the client
ssm = boto3.client('ssm')

class Regions:
    @classmethod
    def get_regions(cls):
        short_codes = cls._get_region_short_codes()
        
        regions = [{
            'name': cls._get_region_long_name(sc),
            'code': sc
        } for sc in short_codes]

        regions_sorted = sorted(
            regions,
            key=lambda k: k['name']
        )

        return regions_sorted

    @classmethod
    def _get_region_long_name(cls, short_code):
        param_name = (
            '/aws/service/global-infrastructure/regions/'
            f'{short_code}/longName'
        )
        response = ssm.get_parameters(
            Names=[param_name]
        )
        return response['Parameters'][0]['Value']

    @classmethod
    def _get_region_short_codes(cls):
        output = set()
        for page in ssm.get_paginator('get_parameters_by_path').paginate(
            Path='/aws/service/global-infrastructure/regions'
        ):
            output.update(p['Value'] for p in page['Parameters'])

        return output

regions = Regions.get_regions()
rn2rc = {r['name']: r['code'] for r in regions}; rn2rc
rc2rn = {r['code']: r['name'] for r in regions}; rc2rn;

def get_price(instance_type, region_name):
    """
    :param instance_type is the type such as 't2.micro'
    :param region_name is the name such as 'US East (N. Virginia)'
    :return price (float)
    """
    response = pricing.get_products(
        ServiceCode='AmazonEC2',
        Filters=[
            {'Type': 'TERM_MATCH','Field': 'operation', 'Value': 'RunInstances'},
            {'Type': 'TERM_MATCH','Field': 'capacitystatus', 'Value': 'Used'},
            {'Type': 'TERM_MATCH','Field': 'operatingSystem', 'Value': 'Linux'},
            {'Type': 'TERM_MATCH', 'Field': 'instanceType', 'Value': instance_type}, #'<insance_type>, e.g. r4.large
            {'Type': 'TERM_MATCH', 'Field': 'location', 'Value': region_name},
        ],
    ); response
    price_item = [json.loads(p) for p in response["PriceList"]]; price_item
    price = float(list(list(price_item[0]['terms']['OnDemand'].values())[0]['priceDimensions'].values())[0]['pricePerUnit']['USD']); price
    price_time_unit = list(list(price_item[0]['terms']['OnDemand'].values())[0]['priceDimensions'].values())[0]['unit']; price_time_unit
    if price_time_unit != 'Hrs': raise Exception('Price given is not in hours')
    return price

# To check for which instances are running and give info on them
def get_instance_details():    
    instance_details = []
    reservations = ec2.describe_instances().get('Reservations'); reservations#.keys()
    instances = [next(iter(i.get('Instances', {}))) for i in reservations]; instances
    for instance in instances:
        i_type = instance.get('InstanceType')
        r_name = [r['name'] for r in regions if r['code'] in instance['Placement']['AvailabilityZone']][0]
        name = next(iter([tag['Value'] for tag in instance.get('Tags') if tag.get('Key')=='Name']), None)#[?Key==`Name`].Value
        state = instance.get('State',{}).get('Name','')
        started = datetime.strftime(instance.get('LaunchTime'), '%y-%m-%d @ %T %Z')
        running = (datetime.now(instance.get('LaunchTime').tzinfo) - instance.get('LaunchTime')).days
        price = get_price(i_type, r_name)
        details = {
            'InstanceType': i_type, 
            'Started':started,
            'RunningDuration':running,
            'State':state,
            'Name':name,
            'Region':r_name,
            'Price':price
        }
        instance_details.append(details); instance_details
    return instance_details

def get_monthly_spend(instance_details):
    """
    :param instance_details comes from get_instance_details and is a list of dicts
    """
    monthly_spend = sum(d['Price']*24*30 for d in instance_details if d['State'] != 'stopped'); monthly_spend
    return monthly_spend

def get_notification_message(instance_details):
    monthly_spend = get_monthly_spend(instance_details);
    notification_message = f'Expected cloud spend is ${round(monthly_spend,2)} / Month'
    for instance in instance_details:
        if instance['State'] != 'running': continue
        i_type = instance['InstanceType']
        price = round(instance['Price'], 3)
        name = instance['Name']
        running = instance['RunningDuration']
        notification_message += f"\n${price}: {i_type}, {name[:10]}- on {running} days"
    return notification_message

def run_server(daily_spend_treshold:int=0, daily_summary_times:list=None, interval_mins:int=60):
    """
    :param daily_spend_treshold (int) number of dollars over which daily spend will be notified
    :param daily_summary_times (int list) 24hr times to log instances
    :param interval_mins is the number of minutes to wait between running this program
    """
    logging.info('Version 1.0 - 08/30/20')
    
    # initialization
    past_notifications = {} # dict of dates and hours

    while True:
        now = datetime.now().astimezone(pytz.timezone('US/Eastern'))
        date = now.strftime('%m-%d'); date
        hour = now.strftime('%H'); hour
        
        # Check if spending is above limit every interval_mins
        instance_details = get_instance_details()
        if get_monthly_spend(instance_details) > daily_spend_treshold:
            notify(f"WARNING: High cloud spend\n {get_notification_message(instance_details)}")

        # If the time is right for a always-notify
        elif hour in daily_summary_times and hour not in past_notifications.setdefault(date, []): 
            # Do the notification
            notify(get_notification_message(instance_details))

            # Store
            past_notifications[date].append(hour)

        pytime.sleep(60 * interval_mins)